In [1]:
import numpy as np
import pandas as pd
import glob
import emcee
import corner
import scipy.stats

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

from fit_just_early_lc import prep_light_curve

from corner_hack import corner_hack
from light_curve_plot import f_t, plot_both_filt

In [2]:
%matplotlib notebook

In [3]:
info_path="../../forced_lightcurves/sample_lc_v2/"
salt_df = pd.read_csv(info_path + "../../Nobs_cut_salt2_spec_subtype_pec.csv")

# Examine effective of fitting first 40% of light  curve

In [4]:
res_25 = pd.read_csv('results_25percent.csv')
res_30 = pd.read_csv('results_30percent.csv')
res_35 = pd.read_csv('results_35percent.csv')
res_40 = pd.read_csv('results_40percent.csv')
res_45 = pd.read_csv('results_45percent.csv')
res_50 = pd.read_csv('results_50percent.csv')

n_good = 0

consistent_list = []
for sn_num, sn in enumerate(res_35.ztf_name):
    if ((res_30['t0'].iloc[sn_num] >= res_35['t0_16'].iloc[sn_num] and 
         res_30['t0'].iloc[sn_num] <= res_35['t0_84'].iloc[sn_num]) and
        (res_30['alpha_g'].iloc[sn_num] >= res_35['alpha_g_16'].iloc[sn_num] and 
         res_30['alpha_g'].iloc[sn_num] <= res_35['alpha_g_84'].iloc[sn_num]) and 
        (res_30['alpha_r'].iloc[sn_num] >= res_35['alpha_r_16'].iloc[sn_num] and 
         res_30['alpha_r'].iloc[sn_num] <= res_35['alpha_r_84'].iloc[sn_num]) and
        (res_40['t0'].iloc[sn_num] >= res_35['t0_16'].iloc[sn_num] and 
         res_40['t0'].iloc[sn_num] <= res_35['t0_84'].iloc[sn_num]) and
        (res_40['alpha_g'].iloc[sn_num] >= res_35['alpha_g_16'].iloc[sn_num] and 
         res_40['alpha_g'].iloc[sn_num] <= res_35['alpha_g_84'].iloc[sn_num]) and 
        (res_40['alpha_r'].iloc[sn_num] >= res_35['alpha_r_16'].iloc[sn_num] and 
         res_40['alpha_r'].iloc[sn_num] <= res_35['alpha_r_84'].iloc[sn_num])
       ):
        consistent_list.append(sn)
        n_good += 1
print(n_good)

63


In [5]:
# check if 5% change is consistent
n_good = 0
consistent_list = []
for sn_num, sn in enumerate(res_35.ztf_name):
    if ((res_25['t0_95'].iloc[sn_num] >= res_30['t0_05'].iloc[sn_num] and 
         res_25['t0_05'].iloc[sn_num] <= res_30['t0_95'].iloc[sn_num]) and
        (res_25['alpha_g_95'].iloc[sn_num] >= res_30['alpha_g_05'].iloc[sn_num] and 
         res_25['alpha_g_05'].iloc[sn_num] <= res_30['alpha_g_95'].iloc[sn_num]) and 
        (res_25['alpha_r_95'].iloc[sn_num] >= res_30['alpha_r_05'].iloc[sn_num] and 
         res_25['alpha_r_05'].iloc[sn_num] <= res_30['alpha_r_95'].iloc[sn_num]) and
        (res_30['t0_95'].iloc[sn_num] >= res_35['t0_05'].iloc[sn_num] and 
         res_30['t0_05'].iloc[sn_num] <= res_35['t0_95'].iloc[sn_num]) and
        (res_30['alpha_g_95'].iloc[sn_num] >= res_35['alpha_g_05'].iloc[sn_num] and 
         res_30['alpha_g_05'].iloc[sn_num] <= res_35['alpha_g_95'].iloc[sn_num]) and 
        (res_30['alpha_r_95'].iloc[sn_num] >= res_35['alpha_r_05'].iloc[sn_num] and 
         res_30['alpha_r_05'].iloc[sn_num] <= res_35['alpha_r_95'].iloc[sn_num]) and
        (res_40['t0_95'].iloc[sn_num] >= res_35['t0_05'].iloc[sn_num] and 
         res_40['t0_05'].iloc[sn_num] <= res_35['t0_95'].iloc[sn_num]) and
        (res_40['alpha_g_95'].iloc[sn_num] >= res_35['alpha_g_05'].iloc[sn_num] and 
         res_40['alpha_g_05'].iloc[sn_num] <= res_35['alpha_g_95'].iloc[sn_num]) and 
        (res_40['alpha_r_95'].iloc[sn_num] >= res_35['alpha_r_05'].iloc[sn_num] and 
         res_40['alpha_r_05'].iloc[sn_num] <= res_35['alpha_r_95'].iloc[sn_num]) and 
        (res_45['t0_95'].iloc[sn_num] >= res_40['t0_05'].iloc[sn_num] and 
         res_45['t0_05'].iloc[sn_num] <= res_40['t0_95'].iloc[sn_num]) and
        (res_45['alpha_g_95'].iloc[sn_num] >= res_40['alpha_g_05'].iloc[sn_num] and 
         res_45['alpha_g_05'].iloc[sn_num] <= res_40['alpha_g_95'].iloc[sn_num]) and 
        (res_45['alpha_r_95'].iloc[sn_num] >= res_40['alpha_r_05'].iloc[sn_num] and 
         res_45['alpha_r_05'].iloc[sn_num] <= res_40['alpha_r_95'].iloc[sn_num]) and
        (res_50['t0_95'].iloc[sn_num] >= res_45['t0_05'].iloc[sn_num] and 
         res_50['t0_05'].iloc[sn_num] <= res_45['t0_95'].iloc[sn_num]) and
        (res_50['alpha_g_95'].iloc[sn_num] >= res_45['alpha_g_05'].iloc[sn_num] and 
         res_50['alpha_g_05'].iloc[sn_num] <= res_45['alpha_g_95'].iloc[sn_num]) and 
        (res_50['alpha_r_95'].iloc[sn_num] >= res_45['alpha_r_05'].iloc[sn_num] and 
         res_50['alpha_r_05'].iloc[sn_num] <= res_45['alpha_r_95'].iloc[sn_num])
       ):
        consistent_list.append(sn)
        n_good += 1

print(n_good)

105


In [6]:
for res_num, this_res in enumerate([res_25, res_30, res_35, res_40, res_45, res_50]):
    n_good = 0
    if res_num == 3:
        consistent_arr = np.zeros(len(res_40))
    for sn_num, sn in enumerate(this_res.ztf_name):
        if ((res_25['t0_84'].iloc[sn_num] >= this_res['t0_16'].iloc[sn_num] and 
             res_25['t0_16'].iloc[sn_num] <= this_res['t0_84'].iloc[sn_num]) and
            (res_25['alpha_g_84'].iloc[sn_num] >= this_res['alpha_g_16'].iloc[sn_num] and 
             res_25['alpha_g_16'].iloc[sn_num] <= this_res['alpha_g_84'].iloc[sn_num]) and 
            (res_25['alpha_r_84'].iloc[sn_num] >= this_res['alpha_r_16'].iloc[sn_num] and 
             res_25['alpha_r_16'].iloc[sn_num] <= this_res['alpha_r_84'].iloc[sn_num]) and
            (res_30['t0_84'].iloc[sn_num] >= this_res['t0_16'].iloc[sn_num] and 
             res_30['t0_16'].iloc[sn_num] <= this_res['t0_84'].iloc[sn_num]) and
            (res_30['alpha_g_84'].iloc[sn_num] >= this_res['alpha_g_16'].iloc[sn_num] and 
             res_30['alpha_g_16'].iloc[sn_num] <= this_res['alpha_g_84'].iloc[sn_num]) and 
            (res_30['alpha_r_84'].iloc[sn_num] >= this_res['alpha_r_16'].iloc[sn_num] and 
             res_30['alpha_r_16'].iloc[sn_num] <= this_res['alpha_r_84'].iloc[sn_num]) and
            (res_35['t0_84'].iloc[sn_num] >= this_res['t0_16'].iloc[sn_num] and 
             res_35['t0_16'].iloc[sn_num] <= this_res['t0_84'].iloc[sn_num]) and
            (res_35['alpha_g_84'].iloc[sn_num] >= this_res['alpha_g_16'].iloc[sn_num] and 
             res_35['alpha_g_16'].iloc[sn_num] <= this_res['alpha_g_84'].iloc[sn_num]) and 
            (res_35['alpha_r_84'].iloc[sn_num] >= this_res['alpha_r_16'].iloc[sn_num] and 
             res_35['alpha_r_16'].iloc[sn_num] <= this_res['alpha_r_84'].iloc[sn_num]) and
            (res_40['t0_84'].iloc[sn_num] >= this_res['t0_16'].iloc[sn_num] and 
             res_40['t0_16'].iloc[sn_num] <= this_res['t0_84'].iloc[sn_num]) and
            (res_40['alpha_g_84'].iloc[sn_num] >= this_res['alpha_g_16'].iloc[sn_num] and 
             res_40['alpha_g_16'].iloc[sn_num] <= this_res['alpha_g_84'].iloc[sn_num]) and 
            (res_40['alpha_r_84'].iloc[sn_num] >= this_res['alpha_r_16'].iloc[sn_num] and 
             res_40['alpha_r_16'].iloc[sn_num] <= this_res['alpha_r_84'].iloc[sn_num]) and
             (res_45['t0_84'].iloc[sn_num] >= this_res['t0_16'].iloc[sn_num] and 
              res_45['t0_16'].iloc[sn_num] <= this_res['t0_84'].iloc[sn_num]) and
             (res_45['alpha_g_84'].iloc[sn_num] >= this_res['alpha_g_16'].iloc[sn_num] and 
              res_45['alpha_g_16'].iloc[sn_num] <= this_res['alpha_g_84'].iloc[sn_num]) and 
             (res_45['alpha_r_84'].iloc[sn_num] >= this_res['alpha_r_16'].iloc[sn_num] and 
              res_45['alpha_r_16'].iloc[sn_num] <= this_res['alpha_r_84'].iloc[sn_num]) and 
              (res_50['t0_84'].iloc[sn_num] >= this_res['t0_16'].iloc[sn_num] and 
               res_50['t0_16'].iloc[sn_num] <= this_res['t0_84'].iloc[sn_num]) and
              (res_50['alpha_g_84'].iloc[sn_num] >= this_res['alpha_g_16'].iloc[sn_num] and 
               res_50['alpha_g_16'].iloc[sn_num] <= this_res['alpha_g_84'].iloc[sn_num]) and 
              (res_50['alpha_r_84'].iloc[sn_num] >= this_res['alpha_r_16'].iloc[sn_num] and 
               res_50['alpha_r_16'].iloc[sn_num] <= this_res['alpha_r_84'].iloc[sn_num])
           ):
            n_good += 1
            if res_num == 3:
                consistent_arr[sn_num] = 1

    print(n_good)

49
61
63
64
55
45


In [7]:
for res_num, this_res in enumerate([res_25, res_30, res_35, res_40, res_45, res_50]):
    n_good = 0
    if res_num == 3:
        consistent_arr = np.zeros(len(res_40))
    for sn_num, sn in enumerate(this_res.ztf_name):
        if ((res_25['t0_95'].iloc[sn_num] >= this_res['t0_05'].iloc[sn_num] and 
             res_25['t0_05'].iloc[sn_num] <= this_res['t0_95'].iloc[sn_num]) and
            (res_25['alpha_g_95'].iloc[sn_num] >= this_res['alpha_g_05'].iloc[sn_num] and 
             res_25['alpha_g_05'].iloc[sn_num] <= this_res['alpha_g_95'].iloc[sn_num]) and 
            (res_25['alpha_r_95'].iloc[sn_num] >= this_res['alpha_r_05'].iloc[sn_num] and 
             res_25['alpha_r_05'].iloc[sn_num] <= this_res['alpha_r_95'].iloc[sn_num]) and
            (res_30['t0_95'].iloc[sn_num] >= this_res['t0_05'].iloc[sn_num] and 
             res_30['t0_05'].iloc[sn_num] <= this_res['t0_95'].iloc[sn_num]) and
            (res_30['alpha_g_95'].iloc[sn_num] >= this_res['alpha_g_05'].iloc[sn_num] and 
             res_30['alpha_g_05'].iloc[sn_num] <= this_res['alpha_g_95'].iloc[sn_num]) and 
            (res_30['alpha_r_95'].iloc[sn_num] >= this_res['alpha_r_05'].iloc[sn_num] and 
             res_30['alpha_r_05'].iloc[sn_num] <= this_res['alpha_r_95'].iloc[sn_num]) and
            (res_35['t0_95'].iloc[sn_num] >= this_res['t0_05'].iloc[sn_num] and 
             res_35['t0_05'].iloc[sn_num] <= this_res['t0_95'].iloc[sn_num]) and
            (res_35['alpha_g_95'].iloc[sn_num] >= this_res['alpha_g_05'].iloc[sn_num] and 
             res_35['alpha_g_05'].iloc[sn_num] <= this_res['alpha_g_95'].iloc[sn_num]) and 
            (res_35['alpha_r_95'].iloc[sn_num] >= this_res['alpha_r_05'].iloc[sn_num] and 
             res_35['alpha_r_05'].iloc[sn_num] <= this_res['alpha_r_95'].iloc[sn_num]) and
            (res_40['t0_95'].iloc[sn_num] >= this_res['t0_05'].iloc[sn_num] and 
             res_40['t0_05'].iloc[sn_num] <= this_res['t0_95'].iloc[sn_num]) and
            (res_40['alpha_g_95'].iloc[sn_num] >= this_res['alpha_g_05'].iloc[sn_num] and 
             res_40['alpha_g_05'].iloc[sn_num] <= this_res['alpha_g_95'].iloc[sn_num]) and 
            (res_40['alpha_r_95'].iloc[sn_num] >= this_res['alpha_r_05'].iloc[sn_num] and 
             res_40['alpha_r_05'].iloc[sn_num] <= this_res['alpha_r_95'].iloc[sn_num]) and
             (res_45['t0_95'].iloc[sn_num] >= this_res['t0_05'].iloc[sn_num] and 
              res_45['t0_05'].iloc[sn_num] <= this_res['t0_95'].iloc[sn_num]) and
             (res_45['alpha_g_95'].iloc[sn_num] >= this_res['alpha_g_05'].iloc[sn_num] and 
              res_45['alpha_g_05'].iloc[sn_num] <= this_res['alpha_g_95'].iloc[sn_num]) and 
             (res_45['alpha_r_95'].iloc[sn_num] >= this_res['alpha_r_05'].iloc[sn_num] and 
              res_45['alpha_r_05'].iloc[sn_num] <= this_res['alpha_r_95'].iloc[sn_num]) and 
              (res_50['t0_95'].iloc[sn_num] >= this_res['t0_05'].iloc[sn_num] and 
               res_50['t0_05'].iloc[sn_num] <= this_res['t0_95'].iloc[sn_num]) and
              (res_50['alpha_g_95'].iloc[sn_num] >= this_res['alpha_g_05'].iloc[sn_num] and 
               res_50['alpha_g_05'].iloc[sn_num] <= this_res['alpha_g_95'].iloc[sn_num]) and 
              (res_50['alpha_r_95'].iloc[sn_num] >= this_res['alpha_r_05'].iloc[sn_num] and 
               res_50['alpha_r_05'].iloc[sn_num] <= this_res['alpha_r_95'].iloc[sn_num])
           ):
            n_good += 1
            if res_num == 3:
                consistent_arr[sn_num] = 1

    print(n_good)

89
95
99
98
93
88


In [8]:
conv_all = (res_25.conv_arr.values*
    res_30.conv_arr.values*
    res_35.conv_arr.values*
    res_40.conv_arr.values*
    res_45.conv_arr.values*
    res_50.conv_arr.values)
print('{} LCs converge in every iteration'.format(sum(conv_all)))
print('{} are consistent by CR_90 definition'.format(sum(consistent_arr)))

115 LCs converge in every iteration
98.0 are consistent by CR_90 definition


In [9]:
salt_df[['name','z_adopt']].iloc[np.where((conv_all == 1) & (consistent_arr == 0))]

,name,z_adopt
0,ZTF18aailmnv,0.080000
14,ZTF18aasdted,0.018150
16,ZTF18aaslhxt,0.055050
21,ZTF18aaunfqq,0.071095
29,ZTF18aaxcntm,0.026922
30,ZTF18aaxdrjn,0.033987
35,ZTF18aaxwjmp,0.084000
49,ZTF18abbpeqo,0.066741
56,ZTF18abclfee,0.029000
72,ZTF18abetehf,0.064875


Notes on this sample -- 

 * ZTF18aailmnv –– 5.5 d gap
 * ZTF18aasdted –– 4 d gap  (followed by 3 d, also low-z)
 * ZTF18aaslhxt –– gap
 * ZTF18aaunfqq –– gap
 * ZTF18aaxcntm –– gaps
 * ZTF18aaxdrjn –– gaps
 * ZTF18aaxwjmp –– small g gaps (this one confuses me)
 * ZTF18abbpeqo –– small g gap
 * ZTF18abclfee –– no gaps
 * ZTF18abetehf –– small gaps
 * ZTF18abfhryc –– small gaps
 * ZTF18abimsyv –– no gaps
 * ZTF18abjtger –– many gaps
 * ZTF18abjvhec –– big gap
 * ZTF18abpaywm –– big g gap (small r gap)
 * ZTF18abxxssh –– weird colors
 * ZTF18abxygvv –– no gaps

### Determine trends with data fit

In [10]:
increase_rise = np.where((res_50.t0.values < res_25.t0.values) &
                         (res_50.t0.values < res_30.t0.values) & 
                         (res_45.t0.values < res_25.t0.values) & 
                         (res_45.t0.values < res_30.t0.values) 
                         )

decrease_rise = np.where((res_50.t0.values > res_25.t0.values) &
                         (res_50.t0.values > res_30.t0.values) & 
                         (res_45.t0.values > res_25.t0.values) & 
                         (res_45.t0.values > res_30.t0.values) 
                         )

In [11]:
print('rise time increases for {} SN with more data'.format(len(np.intersect1d(increase_rise[0], np.where(conv_all > 0)))))
print('rise time decreases for {} SN with less data'.format(len(np.intersect1d(decrease_rise[0], np.where(conv_all > 0)))))

rise time increases for 80 SN with more data
rise time decreases for 17 SN with less data


In [12]:
print('rise time increases for {} SN with more data'.format(len(np.intersect1d(increase_rise[0], np.where(consistent_arr > 0)))))
print('rise time decreases for {} SN with less data'.format(len(np.intersect1d(decrease_rise[0], np.where(consistent_arr > 0)))))

rise time increases for 68 SN with more data
rise time decreases for 16 SN with less data


In [13]:
increase_alpha_g = np.where((res_50.alpha_g.values > res_25.alpha_g.values) &
                         (res_50.alpha_g.values > res_30.alpha_g.values) & 
                         (res_45.alpha_g.values > res_25.alpha_g.values) & 
                         (res_45.alpha_g.values > res_30.alpha_g.values) 
                         )

decrease_alpha_g = np.where((res_50.alpha_g.values < res_25.alpha_g.values) &
                         (res_50.alpha_g.values < res_30.alpha_g.values) & 
                         (res_45.alpha_g.values < res_25.alpha_g.values) & 
                         (res_45.alpha_g.values < res_30.alpha_g.values) 
                         )

In [14]:
print('alpha increases for {} SN with more data'.format(len(np.intersect1d(increase_alpha_g[0], np.where(conv_all > 0)))))
print('alpha decreases for {} SN with less data'.format(len(np.intersect1d(decrease_alpha_g[0], np.where(conv_all > 0)))))

alpha increases for 88 SN with more data
alpha decreases for 7 SN with less data


In [15]:
print('alpha increases for {} SN with more data'.format(len(np.intersect1d(increase_alpha_g[0], np.where(consistent_arr > 0)))))
print('alpha decreases for {} SN with less data'.format(len(np.intersect1d(decrease_alpha_g[0], np.where(consistent_arr > 0)))))

alpha increases for 75 SN with more data
alpha decreases for 7 SN with less data


<IPython.core.display.Javascript object>


In [25]:
color_dict = {'decrease': '#7570b3',
              'increase': '#b2df8a'}

fig, (ax1,ax2) = plt.subplots(2,1,figsize=(4,8), sharex=True)

_ = ax1.plot(haha_rise[:,increase_rise_plot], color='#b2df8a', alpha=0.7, lw=0.8)
_ = ax1.plot(haha_rise[:,decrease_rise_plot], color=color_dict['decrease'], alpha=1, lw=1.5)

ax1.plot([1e6,2e6],[1e6,2e6], 
        color='#b2df8a', alpha=0.7, lw=0.8, 
        label=r'$t_\mathrm{rise}$ increases')
ax1.plot([1e6,2e6],[1e6,2e6], 
        color=color_dict['decrease'], alpha=1, lw=1.5, 
        label=r'$t_\mathrm{rise}$ decreases')
ax1.legend(fancybox=True,fontsize=12)
ax1.set_ylim(-12.4, -27.4)
ax1.set_ylabel(r'$t_\mathrm{fl}$',fontsize=13)

_ = ax2.plot(haha_alpha[:,increase_alpha_plot], color='#b2df8a', alpha=0.7, lw=0.8)
_ = ax2.plot(haha_alpha[:,decrease_alpha_plot], color=color_dict['decrease'], alpha=1, lw=1.5)

ax2.plot([1e6,2e6],[1e6,2e6], 
        color='#b2df8a', alpha=0.7, lw=0.8, 
        label=r'$\alpha_g$ increases')
ax2.plot([1e6,2e6],[1e6,2e6], 
        color=color_dict['decrease'], alpha=1, lw=1.5, 
        label=r'$\alpha_g$ decreases')
ax2.legend(fancybox=True,fontsize=12)
ax2.set_ylim(0,3.75)
ax2.set_ylabel(r'$\alpha_g$',fontsize=13)

ax2.set_xlim(-0.1,5.1)
ax2.set_xticklabels([0.25,0.3,0.35,0.4,0.45,0.5])
ax2.set_xticks([0,1,2,3,4,5])
ax2.set_xlabel(r'$f_\mathrm{thresh}$', fontsize=13)                     
fig.subplots_adjust(left=0.16, right=0.98,
                    bottom=0.07, top=0.99,
                    hspace=0.035)
fig.align_xlabels()
fig.savefig('../paper/figures/flux_frac.pdf')